# Trading Hours, Spreads and Granularity - control and limit Trading Costs

__Goal__: Finding the right Trading hours and granularity to control and limit costs.

__Problem__: We can´t forecast returns with high accuracy -> In all cases where we predict market direction correctly, price movements/volatility must be large enough to cover trading costs.

__Solution__: <br> 
-Make Trades only during busy Trading hours (increased Volatility)<br>
-Lower Granularity leads to larger price movements per bar/candle (to cover Trading Costs) 


__Trade Off__: The higher the Granularity the more likely it is to find markets inefficiencies and mispricing.

## Getting and Preparing the Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("seaborn")

In [ ]:
df = pd.read_csv("bid_ask.csv", parse_dates = ["time"], index_col = "time")

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df.index.tz

In [ ]:
df["NYTime"] = df.index.tz_convert("America/New_York")

In [ ]:
df

In [ ]:
df["hour"] = df.NYTime.dt.hour

In [ ]:
df

In [ ]:
df["price_change_abs"] = df.mid.diff().abs()

In [ ]:
df

In [ ]:
df.dropna(inplace = True)

## The best time to trade (Part 1)

In [ ]:
by_hour = df.groupby("hour")[["volume", "spread", "price_change_abs"]].mean()
by_hour

In [ ]:
by_hour.volume.plot(kind = "bar", figsize = (12, 8), fontsize = 13)
plt.xlabel("NY Time", fontsize = 15)
plt.ylabel("Trading Volumne EUR/USD", fontsize = 15)
plt.title("Trading Volume", fontsize = 15)
plt.show()

In [ ]:
by_hour.spread.plot(kind = "bar", figsize = (12, 8), fontsize = 13)
plt.xlabel("NY Time", fontsize = 15)
plt.ylabel("Spread EUR/USD", fontsize = 15)
plt.title("Spread", fontsize = 15)
plt.show()

In [ ]:
by_hour.loc[0:16, "spread"].plot(kind = "bar", figsize = (12, 8), fontsize = 13)
plt.xlabel("NY Time", fontsize = 15)
plt.ylabel("Spread EUR/USD", fontsize = 15)
plt.title("Spread", fontsize = 15)
plt.show()

In [ ]:
by_hour.price_change_abs.plot(kind = "bar", figsize = (12, 8), fontsize = 13)
plt.xlabel("NY Time", fontsize = 15)
plt.ylabel("Price Changes EUR/USD", fontsize = 15)
plt.title("Price Changes", fontsize = 15)
plt.show()

## The best time to trade (Part 2)

In [ ]:
df

In [ ]:
df["cover_cost"] = df.price_change_abs > df.spread

In [ ]:
df

In [ ]:
df.groupby("hour").cover_cost.mean()

In [ ]:
df.groupby("hour").cover_cost.mean().plot(kind = "bar", figsize = (12, 8), fontsize = 13)
plt.xlabel("NY Time", fontsize = 15)
plt.ylabel("Percentage of Bars where Costs are covered", fontsize = 15)
plt.show()

Busy Trading Hours for __EUR/USD__: from __2:00 am__ to __12:59 pm__ New York Time

## Proportional Trading Costs and Trading Hours

In [ ]:
ptc = 0.00007

In [ ]:
ptc

In [ ]:
df

In [ ]:
df.spread.mean()

In [ ]:
spread = df.loc[df.hour.between(2, 12), "spread"].mean()
spread

In [ ]:
ptc = (spread/2) / df.mid.mean()
ptc

## The Impact of Granularity

In [ ]:
def hours_granularity(freq = None):
    df = pd.read_csv("bid_ask.csv", parse_dates = ["time"], index_col = "time",
                     usecols = ["time", "spread", "mid"])
    if freq is not None:
        df = df.resample(freq).last().dropna()
    df["NYTime"] = df.index.tz_convert("America/New_York")
    df["hour"] = df.NYTime.dt.hour
    df["price_change_abs"] = df.mid.diff().abs()
    df["cover_cost"] = df.price_change_abs > df.spread
    
    df.dropna().groupby("hour").cover_cost.mean().plot(kind = "bar", figsize = (12, 8), fontsize = 13)
    plt.xlabel("NY Time", fontsize = 15)
    plt.ylabel("Cover Costs", fontsize = 15)
    plt.title("Granularity: {}".format(freq), fontsize = 18)
    plt.yticks([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95])
    plt.show()

In [ ]:
hours_granularity(freq = "5min")

In [ ]:
hours_granularity(freq = "10min")

In [ ]:
hours_granularity(freq = "20min")

In [ ]:
hours_granularity(freq = "30min")

In [ ]:
hours_granularity(freq = "1H")